In [1]:
import pandas as pd

In [23]:
df1 = pd.read_csv('movies_tmdb_progres1.csv')
df_tmdb_tmdb_tmdb_tmdb_tmdb_tmdb_tmdb_tmdb2 = pd.read_csv('movies_tmdb_progres2.csv')
df3 = pd.read_csv('movies_tmdb_progres3.csv')

# Concatenar verticalmente
df_tmdb = pd.concat([df1, df2, df3], ignore_index=True)

# Guardar el resultado
df_tmdb.to_csv('dataset_tmdb.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'movies_tmdb_progres1.csv'

In [33]:
df_tmdb = pd.read_csv('dataset_tmdb.csv')
df_tmdb.isnull().sum()

title                       0
vote_average                0
vote_count                  0
release_date                0
revenue                     0
runtime                     0
adult                       0
budget                      0
imdb_id                     0
original_language           0
overview                 7520
popularity                  0
tagline                 78166
genres                      0
production_companies        0
production_countries     5850
spoken_languages         9680
keywords                    0
director                 4052
top_actors               8320
budget_tmdb               792
revenue_tmdb              792
runtime_tmdb              792
dtype: int64

In [7]:
df_tmdb[df_tmdb['vote_count'] == 0.0]

,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,imdb_id,original_language,...,spoken_languages,keywords,director,top_actors,budget_tmdb,revenue_tmdb,runtime_tmdb,overview_clean,title_clean,tagline_clean
86365,de la piste aux étoiles,0.0,0,2023-10-22,0,0,False,0,tt22522248,fr,...,French,"formula one (f1), car racing",manu coeman,"stéphane de groodt, claude lelouch, esteban oc...",0.0,0.0,0.0,,piste aux étoile,
86366,the last american colony,0.0,0,2019-10-18,0,90,False,0,tt11143486,en,...,"English, Spanish","revolution, puerto rico, colonization",bestor cram,"juan segarra palmer, pedro albizu campos, rafa...",0.0,0.0,90.0,puerto rico relic colonization hemisphere terr...,american colony,man revolution
86367,postscript,0.0,0,2016-01-01,0,90,False,3000,tt4931948,en,...,English,"anthology, apocalypse, horror anthology, post-...",mark garvey,"mark garvey, paul woodcock, ian tucknott, emma...",3000.0,0.0,90.0,anthology film set future survivor try stay co...,postscript,future max mad
86368,involuntary wingman,0.0,0,2022-10-01,0,8,False,0,tt22267742,en,...,English,"high school, toronto, canada, screwball comedy...",alex ef,"abasi ekpenyon, ojonugwa usman, aaron macpherson",0.0,0.0,8.0,face threat blackmail year david force owen in...,involuntary wingman,celibate involuntary wingman
86369,ricochet,0.0,0,1961-10-03,0,60,False,0,tt0559627,en,...,English,"accidental death, military training",marc daniels,"dick powell, van heflin, john doucette, john a...",0.0,0.0,60.0,army sergeant blame cause rookie death training,ricochet,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122676,lullaby,0.0,0,2014-10-22,0,0,False,0,tt4071216,en,...,NaN,killer clown,clifford duvernois,"dax spanogle, lindsey shaheen",0.0,0.0,0.0,sylvester killer clown leave trail body victim...,,safe
122677,year of the dragon,0.0,0,1989-02-01,0,58,False,0,tt12007916,et,...,"Estonian, Russian","independence movement, perestroika",andres sööt,,0.0,0.0,58.0,year estonia come surprise symbol allow expres...,year dragon,
122678,the merry mistake,0.0,0,2020-12-25,0,15,False,0,tt13567594,en,...,NaN,"holiday, christmas, short story, short film",louise alston,"geovanni gopradi, jacqi vene, vic polizos",0.0,0.0,15.0,note self ask santa glass year piper mistook m...,merry mistake,
122679,in one hundred years in may,0.0,0,1986-11-27,0,92,False,0,tt0219286,et,...,Estonian,communist,kaljo kiisk,"jüri krjukov, jaan rekkor, arvo kukumägi, sule...",0.0,0.0,92.0,viktor kingissepp head communist party estonia...,year,


In [34]:
import spacy
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import joblib
from scipy import sparse

# Cargar modelo de spaCy sin parser ni NER
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
tqdm.pandas()
# 1. Rellenar nulos y limpiar tagline y keywords
for col in ['keywords', 'tagline', 'title', 'overview', 'top_actors', 'director', 'production_companies']:
    df_tmdb[col] = df_tmdb[col].fillna('').str.strip().str.lower()
# 2. Función general para limpiar texto usando POS y lematización
def clean_text_spacy(text, pos_tags):
    doc = nlp(text)
    return " ".join(
        token.lemma_.lower()  # Usamos la raíz léxica (lemma) en lugar del token original
        for token in doc
        if token.pos_ in pos_tags and token.is_alpha and not token.is_stop
    )
# 3. Limpiar overview: solo NOUN, VERB, PROPN
df_tmdb['overview'] = df_tmdb['overview'].fillna('')
df_tmdb['overview_clean'] = df_tmdb['overview'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN']))
# 4. Limpiar title: NOUN, VERB, PROPN, ADJ
df_tmdb['title'] = df_tmdb['title'].fillna('')
df_tmdb['title_clean'] = df_tmdb['title'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN', 'ADJ']))
df_tmdb['tagline'] = df_tmdb['tagline'].fillna('')
df_tmdb['tagline_clean'] = df_tmdb['tagline'].progress_apply(lambda x: clean_text_spacy(x, ['NOUN', 'VERB', 'PROPN', 'ADJ']))


# 6. Eliminar duplicados de palabras por fila

def remove_duplicates(text):
    return " ".join(dict.fromkeys(text.split()))
for col in ['title_clean', 'overview_clean', 'tagline_clean', 'top_actors', 'director', 'production_companies']:
    df_tmdb[col] = df_tmdb[col].progress_apply(remove_duplicates)


df_tmdb.head()

100%|██████████| 122681/122681 [00:00<00:00, 659034.23it/s]


,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,imdb_id,original_language,...,spoken_languages,keywords,director,top_actors,budget_tmdb,revenue_tmdb,runtime_tmdb,overview_clean,title_clean,tagline_clean
0,inception,8.364,34495,2010-07-15,825532764,148,False,160000000,tt1375666,en,...,"English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",christopher nolan,"leonardo dicaprio, joseph gordon-levitt, ken w...",160000000.0,8.390306e+08,148.0,cobb thief commit espionage infiltrate subcons...,inception,mind scene crime
1,interstellar,8.417,32571,2014-11-05,701729206,169,False,165000000,tt0816692,en,...,English,"rescue, future, spacecraft, race against time,...",christopher nolan,"matthew mcconaughey, anne hathaway, michael ca...",165000000.0,7.466067e+08,169.0,adventure group explorer use discover wormhole...,interstellar,mankind bear earth mean die
2,the dark knight,8.512,30619,2008-07-16,1004558444,152,False,185000000,tt0468569,en,...,"English, Mandarin","joker, sadism, chaos, secret identity, crime f...",christopher nolan,"christian bale, heath ledger, aaron eckhart, m...",185000000.0,1.004558e+09,152.0,batman raise stake war crime help lt jim gordo...,dark knight,welcome world rule
3,avatar,7.573,29815,2009-12-15,2923706026,162,False,237000000,tt0499549,en,...,"English, Spanish","future, society, culture clash, space travel, ...",james cameron,"sam worthington, zoe saldaña, sigourney weaver...",237000000.0,2.923706e+09,162.0,century marine dispatch moon pandora mission t...,avatar,enter world pandora
4,the avengers,7.710,29166,2012-04-25,1518815515,143,False,220000000,tt0848228,en,...,"English, Hindi, Russian","new york city, superhero, shield, based on com...",joss whedon,"robert downey jr., chris evans, mark ruffalo, ...",220000000.0,1.518816e+09,143.0,enemy emerge threaten safety security nick fur...,avenger,assembly require


In [44]:
df_tmdb.sample(10)

,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,imdb_id,original_language,...,spoken_languages,keywords,director,top_actors,budget_tmdb,revenue_tmdb,runtime_tmdb,overview_clean,title_clean,tagline_clean
13177,lion of the desert,7.300,147,1981-04-17,1500000,173,False,35000000,tt0081059,en,...,"English, Arabic, Italian","guerrilla warfare, libya, italy, resistance, w...",moustapha akkad,"anthony quinn, rod steiger, oliver reed, irene...",35000000.0,1502136.0,173.0,movie tell story omar mukhtar rebel fight conq...,lion desert,man honor war
103441,wedding festivities,0.000,0,1913-09-29,0,40,False,0,tt3550098,zh,...,No Language,silent film,zheng zhengqiu,"wang bing-seng, 丁楚鹤, 黄小雅, 顾静鹤, 张双宜, 杨润身, 胡恨生, ...",0.0,0.0,40.0,,wedding festivity,
19839,sheep & wolves: pig deal,6.684,68,2019-01-24,3393975,80,False,0,tt8761814,ru,...,Russian,"friendship, wolf, friends",vladimir nikolaev,"elizaveta boyarskaya, maksim matveev, galina k...",0.0,3393975.0,80.0,united town sheep wolf live life guest turn fo...,sheep wolf pig deal,
24004,the free world,5.989,47,2016-09-23,0,100,False,0,tt3517044,en,...,English,"ex-con, abusive husband, unhappy marriage",jason lew,"boyd holbrook, elisabeth moss, octavia spencer...",0.0,0.0,100.0,follow release stretch prison crime commit mo ...,free world,love set free
49621,bermuda island,3.000,4,2023-01-20,0,0,False,0,tt14926914,en,...,English,"monsters of the wild, tropical island, plane c...",adam werth,"tom sizemore, noel gugliemi, john wells, sarah...",0.0,0.0,86.0,passenger way paradise crash sea find island s...,bermuda island,strand paradise sun go
72080,on the sunny side of the alps,7.000,1,2008-01-01,0,15,False,0,tt1839569,sl,...,Slovenian,short film,janez burger,"samuel camara, aisha ibrahim maiga, dado kebe ...",0.0,0.0,15.0,family live alpine idyll neighbor buy car,sunny alp,
104566,talent agency,0.000,0,1993-01-01,0,85,False,0,tt0106229,it,...,Italian,sleazy giallo,ninì grassia,"saverio vallone, alex damiani, daniela paganin...",0.0,0.0,85.0,james director george owner film studio mornin...,talent agency,
20853,the student of prague,6.000,62,1913-08-22,0,85,False,0,tt0003419,de,...,No Language,"prague, czech republic, mysterious man, silent...",stellan rye,"paul wegener, grete berger, lyda salmonova, jo...",0.0,0.0,85.0,prague bohemia balduin student fall love margi...,student prague,
107624,striker's mountain,0.000,0,1985-11-01,0,99,False,2,tt0090091,en,...,English,"canada, helicopter, mountain, snow, snow skiin...",alan simmonds,"leslie nielsen, august schellenberg, mimi kuzy...",2.0,0.0,99.0,construction conglomerate head millionaire wan...,striker mountain,man mountain
96124,inhumanities,0.000,0,1989-01-01,0,70,False,0,tt0261748,en,...,NaN,"true crime, mondo",harvey keith,marc andrews,0.0,0.0,70.0,compilation newsreel footage death scene,inhumanity,ultimate death experience


In [36]:
df_tmdb.shape

(122681, 26)

In [41]:
df_tmdb.dtypes

title                    object
vote_average            float64
vote_count                int64
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
budget                    int64
imdb_id                  object
original_language        object
overview                 object
popularity              float64
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
director                 object
top_actors               object
budget_tmdb             float64
revenue_tmdb            float64
runtime_tmdb            float64
overview_clean           object
title_clean              object
tagline_clean            object
dtype: object

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import joblib

columnas_vectorizadas = ['title_clean', 'overview_clean', 'tagline_clean', 'keywords', 'top_actors', 'director', 'production_companies']
for col in columnas_vectorizadas:
    print(f"Vectorizando columna: {col}")
    
    texts = df_tmdb[col].fillna('').astype(str)

    # Diagnóstico
    empty_count = (texts.str.strip() == '').sum()
    print(f"  Entradas vacías: {empty_count} de {len(texts)}")
    
    if empty_count == len(texts):
        print(f"  ❌ Todos los textos en {col} están vacíos. Se omite.")
        continue

    # Vectorizar
    vectorizer = TfidfVectorizer(max_features=10000, lowercase=True)
    vocab_vect = vectorizer.fit_transform(texts)

    # Guardar vectores y vectorizador
    
    joblib.dump(vectorizer, f"{col}_vect.pkl")



Vectorizando columna: title_clean
  Entradas vacías: 2721 de 122681
Vectorizando columna: overview_clean
  Entradas vacías: 7531 de 122681
Vectorizando columna: tagline_clean
  Entradas vacías: 78644 de 122681
Vectorizando columna: keywords
  Entradas vacías: 0 de 122681
Vectorizando columna: top_actors
  Entradas vacías: 8320 de 122681
Vectorizando columna: director
  Entradas vacías: 4052 de 122681
Vectorizando columna: production_companies
  Entradas vacías: 0 de 122681


In [46]:
df_tmdb.shape

(122681, 26)

In [ ]:
columnas_vectorizadas = ['title_clean', 'overview_clean', 'tagline_clean', 'keywords', 'top_actors', 'director', 'production_companies']

# 7. Vectorización TF-IDF
for col in columnas_vectorizadas:
    vectorizer = TfidfVectorizer(max_features=10000, lowercase=True)
    vocab_vect = vectorizer.fit_transform(df_tmdb[col].apply(lambda x: ' '.join(x)))
    sparse.save_npz('{col}.npz', vocab_vect)
    



ValueError: empty vocabulary; perhaps the documents only contain stop words

In [9]:
df_tmdb.drop(columns=['overview', 'title', 'tagline', 'keywords', 'overview_clean', 'title_clean', 'tagline_clean', 'top_actors', 'director', 'production_companies'], inplace=True)

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
import pandas as pd
# 1. Función para separar y limpiar strings
def split_and_clean(text):
    if pd.isna(text) or text.strip() == "":
        return []
    return [item.strip() for item in text.split(",") if item.strip()]
# 2. Procesar columnas deseadas
cols_to_encode = ['genres', 'spoken_languages', 'production_countries']
for col in cols_to_encode:
    df_tmdb[col] = df_tmdb[col].apply(split_and_clean)
# 3. Binarizar cada columna con MultiLabelBinarizer
for col in cols_to_encode:
    print(f"Procesando columna: {col}")
    mlb = MultiLabelBinarizer(sparse_output=True)
    encoded = mlb.fit_transform(df_tmdb[col])
    # Guardar binarizador
    joblib.dump(mlb, f"{col}_mlb.pkl")
    # Convertir a DataFrame
    encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded, index=df_tmdb.index, columns=[f"{col}_{cls}" for cls in mlb.classes_])
    # Concatenar con el DataFrame original (puedes eliminar la columna original si no la necesitas)
    df_tmdb = pd.concat([df_tmdb, encoded_df], axis=1)
# 4. (Opcional) Eliminar columnas originales si ya no son necesarias
df_tmdb.drop(columns=cols_to_encode, inplace=True)

Procesando columna: genres
Procesando columna: spoken_languages
Procesando columna: production_countries


In [11]:
df_tmdb.shape

(122681, 430)

In [12]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
lang_encoded = ohe.fit_transform(df_tmdb[['original_language']])
joblib.dump(ohe, "original_language_ohe.pkl")
lang_encoded_df = pd.DataFrame.sparse.from_spmatrix(
    lang_encoded, index=df_tmdb.index, columns=[f"lang_{cat}" for cat in ohe.categories_[0]]
)
df_tmdb = pd.concat([df_tmdb, lang_encoded_df], axis=1)
df_tmdb.drop(columns=['original_language'], inplace=True)  # opcional

In [13]:
df_tmdb['adult'] = df_tmdb['adult'].astype(int)  # True → 1, False → 0

In [14]:
df_tmdb['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors='coerce')
df_tmdb['release_year'] = df_tmdb['release_date'].dt.year.fillna(0).astype(int)
df_tmdb.drop(columns=['release_date'], inplace=True)  

In [15]:
from sklearn.preprocessing import MinMaxScaler
numeric_cols = ['vote_average', 'vote_count', 'runtime_tmdb', 'budget_tmdb', 'popularity', 'release_year']
scaler = MinMaxScaler()
df_tmdb[numeric_cols] = scaler.fit_transform(df_tmdb[numeric_cols])
joblib.dump(scaler, "numeric_minmax_scaler.pkl")

['numeric_minmax_scaler.pkl']

In [24]:
df_tmdb.drop(columns=['imdb_id'], inplace=True)  # opcional, si no necesitas el ID de IMDb

In [25]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122681 entries, 0 to 122680
Columns: 569 entries, vote_average to release_year
dtypes: Sparse[float64, 0](141), Sparse[int64, 0](417), float64(7), int64(4)
memory usage: 17.4 MB


In [28]:
from scipy.sparse import hstack, csr_matrix

# Elimina columnas tipo object (como textos, listas, etc.)
df_numeric = df_tmdb.select_dtypes(include=['number', 'Sparse'])

# Convierte solo las columnas numéricas a sparse
X_num_sparse = csr_matrix(df_numeric.values)


In [29]:
print(X_num_sparse.shape)   # Debería ser (122681, algo)
print(vocab_vect.shape)     # Debería ser (122681, algo)


(122681, 569)
(7, 10000)


In [30]:
vocab_vect

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 47888 stored elements and shape (7, 10000)>

In [27]:
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
X_num_sparse = csr_matrix(df_tmdb)
X_combined = hstack((vocab_vect, X_num_sparse))


ValueError: Mismatching dimensions along axis 0: {122681, 7}

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(X_combined)
joblib.dump(model, 'nearest_neighbors_model.pkl')



In [ ]:
from sklearn.neighbors import NearestNeighbors
import joblib
from tqdm import tqdm
import time  # solo para simular retraso, quitar en producción

# Supongamos que preparar X_combined es un proceso con loop
# Aquí simulo con un loop dummy para mostrar tqdm
for _ in tqdm(range(100), desc='Entrenando NearestNeighbors'):
    time.sleep(0.01)  # simula trabajo pesado

# Aquí ya tienes X_combined preparado, entrenas el modelo normalmente
model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(X_combined)

joblib.dump(model, 'nearest_neighbors_model.pkl')
